# Skywalker algorithm against the two mirror system

In [8]:
SIMULATION = False

goal1 = 220 # Target pixel on first imager. NOTE: Don't do the math for flipped image
goal2 = 258 # Target pixel on second imager. NOTE: Don't do the math for flipped image
first_steps = -10 # Naive first step for the parameters search
tolerances = 20.0 # Tolerance of each target in Pixels
average = 50 # Number of shots to average over
timeout = 600 # Timeout in seconds
log_level = "INFO"

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

goals = [goal1, goal2]

In [9]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

###############
#  pswalker   #
###############
from pswalker.examples  import patch_pims
from pswalker.callbacks import PositionSaving
from pswalker.skywalker import skywalker
from pswalker.skywalker import (homs_RE, homs_system)

In [10]:
"""
Parameters
----------
"""
#Configure logger
log_level = getattr(logging, log_level, None)

#Report invalid logging level
if not isinstance(log_level, int):
    raise ValueError("Invalid log level : {}".format(log_level))

#Create basic configuration
logging.basicConfig(level=log_level,
                format='%(asctime)s - %(levelname)s - %(message)s')

#Instantiate simulation
if SIMULATION:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', x=0.0317324, z=101.843, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise    
else:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3


********************************************************************************
* WARNING: Running on real devices in Production...
********************************************************************************
Press any key to continue...


In [11]:
#Create Skywalker plan
# NOTE : the image is rotated 90 degrees, this means we need to be aligning to centroid_y
plan = skywalker([y1, y2], [m1, m2], 'detector_stats2_centroid_y', 'pitch',
             goals, first_steps=first_steps, tolerances=tolerances,
             averages=average, timeout=timeout)


In [12]:
#Create RunEngine
RE = None
if SIMULATION:
    RE = RunEngine({})
    collector = MsgCollector()
    RE.msg_hook = collector
else:
    RE = homs_RE()

In [13]:
RE(run_wrapper(plan), PositionSaving)

2017-07-26 17:40:14,431 - INFO - Waiting for hx2_states_state to be set from 'OUT' to 'YAG'...
2017-07-26 17:40:14,443 - INFO - Waiting for hx2_states_state to be set from 'OUT' to 'YAG'...
2017-07-26 17:40:14,467 - INFO - Waiting for hx2_states_state to be set from 'OUT' to 'YAG'...
2017-07-26 17:40:14,510 - INFO - Waiting for hx2_states_state to be set from 'OUT' to 'YAG'...
2017-07-26 17:40:14,592 - INFO - Waiting for hx2_states_state to be set from 'OUT' to 'YAG'...
2017-07-26 17:40:14,754 - INFO - Waiting for hx2_states_state to be set from 'OUT' to 'YAG'...
2017-07-26 17:40:14,917 - INFO - Waiting for hx2_states_state to be set from 'Unknown' to 'YAG'...
2017-07-26 17:40:15,082 - INFO - Waiting for hx2_states_state to be set from 'Unknown' to 'YAG'...
2017-07-26 17:40:15,244 - INFO - Waiting for hx2_states_state to be set from 'Unknown' to 'YAG'...
2017-07-26 17:40:15,409 - INFO - Waiting for hx2_states_state to be set from 'Unknown' to 'YAG'...
2017-07-26 17:40:15,571 - INFO - W

+-----------+------------+------------+------------+--------------------------------+--------------------------------+
|   seq_num |       time |  m1h_pitch |  m2h_pitch | hx2_detector_stats2_centroid_y | dg3_detector_stats2_centroid_y |
+-----------+------------+------------+------------+--------------------------------+--------------------------------+
|         1 | 17:40:17.9 |   267.8325 |   130.1391 |                          262.6 |                            0.0 |
|         2 | 17:40:17.9 |   267.7272 |   130.1488 |                          260.2 |                            0.0 |
|         3 | 17:40:17.9 |   267.7643 |   130.1488 |                          255.2 |                            0.0 |
|         4 | 17:40:17.9 |   267.7643 |   130.1488 |                          258.0 |                            0.0 |
|         5 | 17:40:18.0 |   267.7643 |   130.1254 |                          255.4 |                            0.0 |
|         6 | 17:40:18.2 |   267.7993 |   130.12

2017-07-26 17:40:29,227 - INFO - Averaged data yielded hx2_detector_stats2_centroid_y is at 152.35550094067355
2017-07-26 17:40:29,229 - INFO - Unable to yield estimate from model Model(Linear)
2017-07-26 17:40:29,231 - INFO - Walking motor, error after step #0 -> 67.64449905932645
2017-07-26 17:40:29,233 - INFO - No model yielded accurate prediction, using naive plan


|        98 | 17:40:29.1 |   267.8130 |   130.0826 |                          257.9 |                            0.0 |
|        99 | 17:40:29.1 |   267.7818 |   130.0923 |                          264.1 |                            0.0 |
|       100 | 17:40:29.2 |   267.7838 |   130.0923 |                          262.4 |                            0.0 |
|       101 | 17:40:41.0 |   257.8856 |   130.1274 |                          343.9 |                            0.0 |
|       102 | 17:40:41.1 |   257.8017 |   130.1274 |                          342.8 |                            0.0 |
|       103 | 17:40:41.1 |   257.8017 |   130.1274 |                          342.9 |                            0.0 |
|       104 | 17:40:41.1 |   257.8017 |   130.1274 |                          338.3 |                            0.0 |
|       105 | 17:40:41.1 |   257.8017 |   130.1274 |                          346.1 |                            0.0 |
|       106 | 17:40:41.2 |   257.8017 |   130.13

2017-07-26 17:40:43,324 - INFO - Averaged data yielded hx2_detector_stats2_centroid_y is at 342.0653735009354
2017-07-26 17:40:43,328 - INFO - Walking motor, error after step #1 -> -122.06537350093538
2017-07-26 17:40:43,330 - INFO - Using model Model(Linear) to determine next step.
2017-07-26 17:40:43,334 - INFO - Adjusting motor m1h to position 265.2765486778542


...

2017-07-26 17:41:53,284 - INFO - Averaged data yielded hx2_detector_stats2_centroid_y is at 246.4673733544352
2017-07-26 17:41:53,288 - INFO - Walking motor, error after step #9 -> -26.46737335443521
2017-07-26 17:41:53,291 - INFO - Using model Model(Linear) to determine next step.
2017-07-26 17:41:53,293 - INFO - Adjusting motor m1h to position 269.1612439191565


|       550 | 17:41:53.2 |   268.6808 |   130.1547 |                          246.6 |                            0.0 |
|       551 | 17:41:59.3 |   269.1780 |   130.1118 |                          245.4 |                            0.0 |
|       552 | 17:41:59.3 |   269.1780 |   130.1118 |                          239.5 |                            0.0 |
|       553 | 17:41:59.3 |   269.1780 |   130.1469 |                          247.9 |                            0.0 |
|       554 | 17:41:59.4 |   269.1312 |   130.1469 |                          242.2 |                            0.0 |
|       555 | 17:41:59.4 |   269.1819 |   130.1098 |                          242.0 |                            0.0 |
|       556 | 17:41:59.5 |   269.1819 |   130.1098 |                          242.2 |                            0.0 |
|       557 | 17:41:59.5 |   269.1819 |   130.1098 |                          242.6 |                            0.0 |
|       558 | 17:41:59.5 |   269.1819 |   130.06

2017-07-26 17:42:01,535 - INFO - Averaged data yielded hx2_detector_stats2_centroid_y is at 243.63607690305764
2017-07-26 17:42:01,539 - INFO - Walking motor, error after step #10 -> -23.636076903057642


|       597 | 17:42:01.4 |   269.1429 |   130.0845 |                          250.1 |                            0.0 |
|       598 | 17:42:01.4 |   269.2365 |   130.1430 |                          243.7 |                            0.0 |
|       599 | 17:42:01.5 |   269.2365 |   130.1430 |                          249.9 |                            0.0 |
|       600 | 17:42:01.5 |   269.2365 |   130.1430 |                          246.0 |                            0.0 |
+-----------+------------+------------+------------+--------------------------------+--------------------------------+
generator run_wrapper ['830951'] (scan num: 1)


RuntimeError: fitwalk failed to converge after 10 steps

In [ ]:
if SIMULATION:
    #Analyze Performance
    print(len(RE.msg_hook.msgs))